In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# url of Ist page

url = 'https://www.themoviedb.org/movie?page=' + str(1)
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
response = requests.get(url, headers = headers)

In [3]:
# parsing
doc = BeautifulSoup(response.text, 'html.parser')

In [4]:
# getting name of the movie

Name_tags = doc.find_all('h2')
Movies_names = []
for i in range (4, len(Name_tags)) :
     Movies_names.append(Name_tags[i].text)

In [5]:
# release year of movie

release_tags = doc.find_all('p')
release_year = []
for i in range (1, 21):
     release_year.append(release_tags[i].text)

In [6]:
# Link of movie

link_tags = doc.find_all('a', {'class' : 'image'})
movie_links = []
base_url ='https://www.themoviedb.org'
for i in range (len(link_tags)):
 
 movie_links.append(base_url + link_tags[i]['href'])

In [7]:
# rating of movie

rating = []
rating_tag = doc.find_all('div', {'class' : 'user_score_chart'})
for i in range (len(rating_tag)):
     rating.append(rating_tag[i]['data-percent'])

In [8]:
# Following same procedure as above for 500 pages

for i in range(2,501):
    
    url = 'https://www.themoviedb.org/movie?page=' + str(i)
    
    response = requests.get(url, headers = headers)
    
    doc = BeautifulSoup(response.text, 'html.parser')
    
    Name_tags = doc.find_all('h2')
    for i in range (4, len(Name_tags)) :
         Movies_names.append(Name_tags[i].text)
            
    release_tags = doc.find_all('p')
    for i in range (1, 21):
         release_year.append(release_tags[i].text)
            
    link_tags = doc.find_all('a', {'class' : 'image'})
    base_url ='https://www.themoviedb.org'
    for i in range (len(link_tags)):
         movie_links.append(base_url + link_tags[i]['href'])
            
    rating_tag = doc.find_all('div', {'class' : 'user_score_chart'})
    for i in range (len(rating_tag)):
         rating.append(rating_tag[i]['data-percent'])

In [9]:
len(movie_links)

10000

# Collecting Data about each movie using Movie links

In [10]:
Movie_url = movie_links[0]
r = requests.get(Movie_url, headers = headers)
movie_docs = BeautifulSoup(r.text, 'html.parser')

In [11]:
# desc about movie
desc_tag = movie_docs.find_all('div', {'class' : 'overview'})
desc = desc_tag[0].text.strip()

In [12]:
# Director of movie
director_tag = movie_docs.find_all('li', {'class' : 'profile'})
Director = director_tag[0].text.strip().split('\n')[0]

In [13]:
# Movie Runtime
runtime_tag = movie_docs.find_all('span', {'class' : 'runtime'})
runtime = runtime_tag[0].text.strip()

In [14]:
# Budget of movie
side_column_tags = movie_docs.find('section', {'class' : 'facts left_column'}).find_all('p')
Budget = side_column_tags[len(side_column_tags)-2].text.strip().split(' ')[1]

In [15]:
# Revenue of movie
Revenue = side_column_tags[len(side_column_tags)-1].text.strip().split(' ')[1]

In [16]:
# Language of movie
Language = side_column_tags[len(side_column_tags)-3].text.strip().split(' ')[2]

In [17]:
#genres
genre_tags = movie_docs.find('span', {'class' : 'genres'}).find_all('a')
for x in range(len(genre_tags)):
    print(genre_tags[x].text)

Action
Science Fiction
Thriller


In [18]:
Description = []
Movie_runtime = []
Budget = []
Revenue = []
Language = []
genres = []
final_genre = []

In [19]:
for i in range (0,10000) :
 
         Movie_url = movie_links[i]
 
         r = requests.get(Movie_url, headers = headers)
 
         movie_docs = BeautifulSoup(r.text, 'html.parser')
        
         try:
            desc_tag = movie_docs.find_all('div', {'class' : 'overview'})
            Description.append(desc_tag[0].text.strip())
         except:
            Description.append(np.NaN)
            
         try:
            runtime_tag = movie_docs.find_all('span', {'class' : 'runtime'})
            Movie_runtime.append(runtime_tag[0].text.strip())
         except:
            Movie_runtime.append(np.NaN)
            
         try:
            side_column_tags = movie_docs.find('section', {'class' : 'facts left_column'}).find_all('p')
            Budget.append(side_column_tags[len(side_column_tags)-2].text.strip().split(' ')[1])
         except:
            Budget.append(np.NaN)
        
         try:
            Revenue.append(side_column_tags[len(side_column_tags)-1].text.strip().split(' ')[1])   
         except:
            Revenue.append(np.NaN)
            
         try:    
            Language.append(side_column_tags[len(side_column_tags)-3].text.strip().split(' ')[2])        
         except:
            Language.append(np.NaN)
            
         try:
            genre_tags = movie_docs.find('span', {'class' : 'genres'}).find_all('a')
            for x in range(len(genre_tags)):
                genres.append(genre_tags[x].text)
         except:
            genres.append(np.NaN)
                
         final_genre.append(genres)
         genres = []


In [20]:
dict = {
    'Movie name' : Movies_names,
    'Release date' : release_year,
    'Language' : Language,
    'Rating' : rating,
    'Runtime' : Movie_runtime,
    'Description' : Description,
    'Budget' : Budget,
    'Revenue' : Revenue,
    'Movie link' : movie_links,
    'Genre' : final_genre,
}

data = pd.DataFrame(dict)

In [21]:
data

,Movie name,Release date,Language,Rating,Runtime,Description,Budget,Revenue,Movie link,Genre
0,Prey,"Aug 02, 2022",English,81.0,1h 40m,"When danger threatens her camp, the fierce and...",-,-,https://www.themoviedb.org/movie/766507,"[Action, Science Fiction, Thriller]"
1,Thor: Love and Thunder,"Jul 06, 2022",English,68.0,1h 59m,After his retirement is interrupted by Gorr th...,"$250,000,000.00","$720,000,000.00",https://www.themoviedb.org/movie/616037,"[Action, Adventure, Fantasy]"
2,Dragon Ball Super: Super Hero,"Jun 11, 2022",Japanese,75.0,1h 39m,"The Red Ribbon Army, an evil organization that...",-,"$17,030,000.00",https://www.themoviedb.org/movie/610150,"[Animation, Science Fiction, Action]"
3,Jurassic World Dominion,"Jun 01, 2022",English,71.0,2h 27m,"Four years after Isla Nublar was destroyed, di...","$165,000,000.00","$974,000,000.00",https://www.themoviedb.org/movie/507086,"[Adventure, Action, Science Fiction]"
4,Top Gun: Maverick,"May 24, 2022",English,83.0,2h 11m,After more than thirty years of service as one...,"$170,000,000.00","$1,381,724,834.00",https://www.themoviedb.org/movie/361743,"[Action, Drama]"
...,...,...,...,...,...,...,...,...,...,...
9995,Man with a Movie Camera,"Jan 08, 1929",Russian,79.0,1h 8m,"A cameraman wanders around Moscow, Kharkiv, Ky...",-,-,https://www.themoviedb.org/movie/26317,[Documentary]
9996,Yojimbo,"Apr 25, 1961",Japanese,82.0,1h 50m,"A nameless ronin, or samurai with no master, e...",-,-,https://www.themoviedb.org/movie/11878,"[Drama, Thriller]"
9997,Evil Angels,"Nov 03, 1988",English,67.0,2h,Based on the true story of Lindy Chamberlain w...,"$15,000,000.00","$6,908,797.00",https://www.themoviedb.org/movie/35119,[Drama]
9998,"Woodstock 99: Peace, Love, and Rage","Jul 23, 2021",English,64.0,1h 50m,"Explore Woodstock 99, a three-day music festiv...",-,-,https://www.themoviedb.org/movie/848562,"[Documentary, Music]"


In [22]:
data.to_csv("Movies Dataset.csv")